# ReMKiT1D input generator - Poisson solver

This notebook creates the json input file for a fluid model with a poisson solver.
The model being solved is a isothermal ion fluid model with Boltzmann distributed electrons:
$$
\frac{\partial n_i}{\partial t} + \nabla \cdot\left( \Gamma_i \right) = S_i
$$
$$
m_i \left(\frac{\partial \mathbf{\Gamma}_i}{\partial t} + \nabla \cdot\left( \mathbf{u}_i \Gamma_i  \right)\right) = -\nabla (n_i k T) - Z e n_i\nabla \phi
$$
$$
n_e = n_0\exp{\left( \frac{e \phi}{T} \right)}
$$
$$
\nabla^2\phi = \frac{e}{\epsilon_0}\left( Z n_i - n_e \right)
$$


In [15]:
import numpy as np
import xarray as xr
import sys

import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

sys.path.append('../')
import RMK_support.simple_containers as sc
import RMK_support.common_models as cm
import RMK_support.IO_support as io
from RMK_support import RKWrapper ,Grid
import RMK_support.sk_normalization as skn
import RMK_support.dashboard_support as ds
from RMK_support import exp, treeDerivation, Node

### Useful constants

In [16]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
amu = 1.6605390666e-27  # atomic mass unit
ionMass = 2.014*amu  # deuterium mass
epsilon0 = 8.854188e-12  # vacuum permittivity
heavySpeciesMass = 2.014  # in amus

### Wrapper initialisation

In [17]:
rk = RKWrapper()

### Global parameters for IO files

In [18]:
rk.jsonFilepath = "./config.json"  # Default value
hdf5Filepath = "./RMKOutput/RMK_poisson_fluid/"
rk.setHDF5Path(hdf5Filepath)

### Setting options for external libraries used by ReMKiT1D

### MPI

In [19]:
numProcsX = 1  # Number of processes in x direction
numProcsH = 1  # Number of processes in harmonic
numProcs = numProcsX * numProcsH
haloWidth = 1  # Halo width in cells

rk.setMPIData(numProcsX, numProcsH, haloWidth)

### Normalisation

In [20]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

# for convenience
tempNorm = rk.normalization["eVTemperature"]
densNorm = rk.normalization["density"]

norms = skn.calculateNorms(tempNorm,densNorm,1.0)

timeNorm = norms["time"]
sigmaNorm = norms["crossSection"]
lengthNorm = norms["length"]
speedNorm = norms["speed"]

phiNorm = elCharge*densNorm*lengthNorm**2/epsilon0

In [21]:
phiNorm

3318555151.8332214

### Grid initialisation

In [22]:
dx0 = 2*0.27/4
dxN = 2*0.0125/4
Nx = 64*4
L = 10 # Length in meters
xGridWidths = L/Nx*np.ones(Nx)
dv0 = 0.05
dvN = 0.4
Nv = 80
vGridWidths = np.geomspace(dv0, dvN, Nv)
lMax = 0
gridObj = Grid(xGridWidths, vGridWidths, lMax, interpretXGridAsWidths=True,
               interpretVGridAsWidths=True, isLengthInMeters=True)

In [23]:
# Add the grid to the config file
rk.grid = gridObj

### Naming Species

In [24]:
rk.addSpecies("e", 0, atomicA=elMass/amu, charge=-1.0, associatedVars=["ne","Ge"])
rk.addSpecies("D+", -1, atomicA=2.014, charge=1.0, associatedVars=["ni", "Gi"])

### Creating initial density configuration

In [25]:
phi = np.zeros(gridObj.numX())
n = np.ones(gridObj.numX())

T = np.ones(gridObj.numX())

### Creating variables

In [26]:
# Units are not used by ReMKiT1D, but are useful to specify for later plotting
rk.addVarAndDual("ni", n, units='$10^{19} m^{-3}$', isCommunicated=True)
rk.addVarAndDual("ne", n, units='$10^{19} m^{-3}$', isCommunicated=True)
rk.addVarAndDual("Gi", primaryOnDualGrid=True, isCommunicated=True)
rk.addVarAndDual("Ge", primaryOnDualGrid=True, isCommunicated=True)

# Temperatures
rk.addVarAndDual("T", T, units='$10eV$', isCommunicated=True)

# Electric potential
rk.addVar("phi", phi, isCommunicated=True, isStationary=True)

# Set derived quantities
rk.addVarAndDual("ui", isDerived=True, primaryOnDualGrid=True,
                 derivationRule=sc.derivationRule("flowSpeedFromFlux", ["Gi_dual", "ni_dual"]), isCommunicated=True)
rk.addVarAndDual("ue", isDerived=True, primaryOnDualGrid=True,
                 derivationRule=sc.derivationRule("flowSpeedFromFlux", ["Ge_dual", "ne_dual"]), isCommunicated=True)
rk.addVar("cs", isDerived=True, derivationRule=sc.derivationRule("sonicSpeedD+", ["T", "T"]))

# Set scalar quantities
rk.addVar("time", isScalar=True, isDerived=True)
rk.addVar("potentialDrop", -3*np.ones(1), isDerived=True,isScalar=True)

### Electron continuity equation

In [27]:
# Electron continuity advection

# Adding the model tag to tag list
modelTag = "continuity-ne"

# Initializing model using common models
elecContModel = cm.staggeredAdvection(modelTag=modelTag, advectedVar="ne",
                                     fluxVar="Ge_dual", advectionSpeed="ue", lowerBoundVar="cs", rightOutflow=True)

rk.addModel(elecContModel.dict())

### Ion continuity equation

In [28]:
# Ion continuity advection

# Adding the model tag to tag list
modelTag = "continuity-ni"

# Initializing model using common models
ionContModel = cm.staggeredAdvection(modelTag=modelTag, advectedVar="ni",
                                     fluxVar="Gi_dual", advectionSpeed="ui", lowerBoundVar="cs", rightOutflow=True)

rk.addModel(ionContModel.dict())

### Pressure gradient forces

In [29]:
# Ion pressure grad

# Adding the model tag to tag list
modelTag = "pressureGrad-Gi"

# Initializing model
ionPressureGradModel = cm.staggeredPressureGrad(modelTag=modelTag, fluxVar="Gi_dual",
                                                densityVar="ni", temperatureVar="T", speciesMass=ionMass)

rk.addModel(ionPressureGradModel.dict())

In [30]:
# Electron pressure grad

# Adding the model tag to tag list
modelTag = "pressureGrad-Ge"

# Initializing model
elecPressureGradModel = cm.staggeredPressureGrad(modelTag=modelTag, fluxVar="Ge_dual",
                                                densityVar="ne", temperatureVar="T", speciesMass=elMass)

rk.addModel(elecPressureGradModel.dict())

### Momentum advection

In [31]:
# Ion momentum advection

# Adding the model tag to tag list
modelTag = "advection-Gi"

# Initializing model
ionMomAdvModel = cm.staggeredAdvection(modelTag=modelTag, advectedVar="Gi_dual", fluxVar="", advectionSpeed="ui", staggeredAdvectionSpeed="ui_dual", lowerBoundVar="cs", rightOutflow=True,
                                       staggeredAdvectedVar=True)

rk.addModel(ionMomAdvModel.dict())


In [32]:
# Electron momentum advection

# Adding the model tag to tag list
modelTag = "advection-Ge"

# Initializing model
elecMomAdvModel = cm.staggeredAdvection(modelTag=modelTag, advectedVar="Ge_dual", fluxVar="", advectionSpeed="ue", staggeredAdvectionSpeed="ue_dual", lowerBoundVar="cs", rightOutflow=True,
                                       staggeredAdvectedVar=True)

rk.addModel(elecMomAdvModel.dict())

### Lorentz term in ion momentum equation

In [33]:
# Ion Lorentz force

# Adding the model tag to tag list
modelTag = "ionLorentz"

# Initializing model
ionLorentzModel = sc.CustomModel(modelTag=modelTag)

evolvedVar = "Gi_dual"

gradPhi = sc.GeneralMatrixTerm(evolvedVar, implicitVar="phi", stencilData=sc.staggeredGradStencil(), customNormConst=phiNorm*elMass/(2*ionMass*tempNorm))

ionLorentzModel.addTerm("gradPhi",gradPhi)

rk.addModel(ionLorentzModel.dict())

In [34]:
# Electron Lorentz force

# Adding the model tag to tag list
modelTag = "elecLorentz"

# Initializing model
elecLorentzModel = sc.CustomModel(modelTag=modelTag)

evolvedVar = "Ge_dual"

gradPhi = sc.GeneralMatrixTerm(evolvedVar, implicitVar="phi", stencilData=sc.staggeredGradStencil(), customNormConst=-phiNorm/(2*tempNorm))

elecLorentzModel.addTerm("gradPhi",gradPhi)

rk.addModel(elecLorentzModel.dict())

### Creating Poisson equation model

In [35]:
# Adding the model tag to tag list
modelTag = "poisson"

normConst = elCharge*densNorm*lengthNorm**2/(epsilon0*phiNorm)

# Initializing model
poissonModel = sc.CustomModel(modelTag=modelTag)

evolvedVar = "phi"

# Adding first laplacian term from poisson's eq.
laplacian = sc.GeneralMatrixTerm(evolvedVar, customNormConst=-1, stencilData=sc.diffusionStencil("none",[]))

poissonModel.addTerm("laplacian",laplacian)

# Adding charge distribution term
chargeDisti = sc.GeneralMatrixTerm(evolvedVar, implicitVar="ni",customNormConst=normConst, stencilData=sc.diagonalStencil())

poissonModel.addTerm("chargeDisti",chargeDisti)

chargeDiste = sc.GeneralMatrixTerm(evolvedVar, implicitVar="ne",customNormConst=-normConst,stencilData=sc.diagonalStencil())

poissonModel.addTerm("chargeDiste",chargeDiste)

# Defining boundary terms

# Calculating
dxN = xGridWidths[-1]/lengthNorm

rightBCPhi = sc.GeneralMatrixTerm(evolvedVar, customNormConst=dxN**-2,
                                  stencilData=sc.diagonalStencil([Nx]))

poissonModel.addTerm("rightBCPhi", rightBCPhi)


rightBCPhiRight = sc.GeneralMatrixTerm(evolvedVar, varData=sc.VarData(["potentialDrop"]),customNormConst=-tempNorm/(phiNorm*dxN**2),
                                       stencilData=sc.diagonalStencil([Nx]),implicitVar="T")

poissonModel.addTerm("rightBCPhiRight", rightBCPhiRight)

dx0 = xGridWidths[0]/lengthNorm

leftBCPhi = sc.GeneralMatrixTerm(evolvedVar, customNormConst=-dx0**-2,
                                 stencilData=sc.diagonalStencil([1]))

poissonModel.addTerm("leftBCPhi", leftBCPhi)

rk.addModel(poissonModel.dict())

### Ion source

In [36]:
# all ionization in last cell
particleInjectionRate = 0.02
xProfileIonization = np.zeros(len(gridObj.xWidths))
xProfileIonization[0]=1
xProfileIonization = particleInjectionRate*xProfileIonization/(sum(xProfileIonization*gridObj.xWidths/lengthNorm))

# Particle source model

# Adding the model tag to tag list
modelTag = "particleSource"

# Initializing model
particleSourceModel = sc.CustomModel(modelTag=modelTag)

# Ions 
evolvedVar = "ni"
particleSourceTermIon = cm.simpleSourceTerm(evolvedVar=evolvedVar,sourceProfile=xProfileIonization)

particleSourceModel.addTerm("ionSource",particleSourceTermIon)

# Electrons 
evolvedVar = "ne"
particleSourceTermIon = cm.simpleSourceTerm(evolvedVar=evolvedVar,sourceProfile=xProfileIonization)

particleSourceModel.addTerm("elecSource",particleSourceTermIon)

rk.addModel(particleSourceModel.dict())

### Integrator options

In [37]:
integrator = sc.picardBDEIntegrator(absTol=10.0,convergenceVars=["phi"],internalStepControl=True)

rk.addIntegrator("BE",integrator)

initialTimestep=0.1
rk.setIntegratorGlobalData(initialTimestep=initialTimestep)

bdeStep = sc.IntegrationStep("BE",defaultUpdateModelData=True)

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

rk.setFixedNumTimesteps(1)
rk.setFixedStepOutput(1)

rk.writeConfigFile()

### Analysis

In [757]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

In [758]:
numFiles = 1

loadFilenames = [hdf5Filepath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)

loadedData

<xarray.Dataset>
Dimensions:                 (x: 256, h: 1, v: 80, time: 2, dim_0: 1)
Coordinates:
  * x                       (x) float64 0.01953 0.05859 0.09766 ... 9.941 9.98
  * h                       (h) int64 0
  * v                       (v) float64 0.025 0.07567 0.1277 ... 12.93 13.32
  * time                    (time) float64 0.0 0.0
Dimensions without coordinates: dim_0
Data variables: (12/23)
    ni                      (time, x) float64 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0
    ni_dual                 (time, x) float64 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0
    ne                      (time, x) float64 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0
    ne_dual                 (time, x) float64 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0
    Gi_dual                 (time, x) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    Gi                      (time, x) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                      ...
    poissonlaplacian        (time, x) float64 0.0 0.0 0.0 ... 0.0 -0.1414
    poissonchargeDisti      (time, x) float64 0.0 0.0 ... 3.319e+08 3.319e+08
    poissonchargeDiste      (time, x) float64 0.0 0.0 ... -3.319e+08 -3.319e+08
    poissonrightBCPhi       (time, x) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 -35.92
    poissonrightBCPhiRight  (time, x) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 36.06
    poissonleftBCPhi        (time, x) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,gridObj)

dashboard.fluid2Comparison().show()